In [8]:
import tweepy
import pandas as pd

api_key = "0esCMwl2tRCZHbQHaK9mkSGsk"
api_key_secret = "TaPrkyToF49zd50GCzpXfNrxPQvUE1tyq1Xd0ERkwYfTr6cYwi"
access_token = "746162773441208320-V43Mv8s1ARVPgMJ9UscAUlJYWQCY5lH"
access_token_secret = "xnrwFtwNPciT0vHPG9KS7SpujXWh2brvrkBfaX7Qxi3qe"
bearer_token = "AAAAAAAAAAAAAAAAAAAAAAHiugEAAAAAGGo7klF3LY0L%2BcCqpkJ%2BOkt7kkM%3DRN08i4EizLBl86DPcejP4TOhp5MssIvZduEWDtKjO6SjHN6RUI"

# Authenticate to Twitter
auth = tweepy.OAuth1UserHandler(api_key, api_key_secret, access_token, access_token_secret)
# api = tweepy.API(auth, wait_on_rate_limit=True)
client = tweepy.Client(bearer_token=bearer_token, consumer_key=api_key, consumer_secret=api_key_secret, access_token=access_token, access_token_secret=access_token_secret)

def scrape_tweets(stock_symbols, max_tweets=5):
    tweets_data = []
    for symbol in stock_symbols:
        query = f"#{symbol} OR {symbol}"
        tweets = client.search_recent_tweets(query=query, tweet_fields=['created_at', 'public_metrics'], max_results=max_tweets)
        for tweet in tweets.data:
            tweets_data.append([
                symbol,
                tweet.created_at,
                tweet.text,
                tweet.public_metrics['like_count'],
                tweet.public_metrics['retweet_count']
            ])
    
    df = pd.DataFrame(tweets_data, columns=['Stock Symbol', 'Timestamp', 'Tweet', 'Likes', 'Retweets'])
    return df

stock_symbols = ['NVDA', 'AMZN', 'EMR', 'MSFT', 'UNH', 'DAL', 'LW', 'ARE', 'MDLZ', 'ELV']
tweets_df = scrape_tweets(stock_symbols, max_tweets=100)
print(tweets_df.head())

TweepyException: This method does not perform pagination

In [ ]:
import requests

def scrape_news(stock_symbols):
    news_data = []
    for symbol in stock_symbols:
        query = f"{symbol} news"
        url = f"https://www.google.com/search?q={query}&tbm=nws"
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        
        for item in soup.find_all('div', attrs={'class': 'BVG0Nb'}):
            headline = item.find('div', attrs={'class': 'BNeawe vvjwJb AP7Wnd'}).text
            link = item.find('a')['href']
            news_data.append([symbol, headline, link])
    
    df = pd.DataFrame(news_data, columns=['Stock Symbol', 'Headline', 'Link'])
    return df

news_df = scrape_news(stock_symbols)
print(news_df.head())

In [ ]:
combined_df = tweets_df.merge(news_df, on='Stock Symbol', how='outer')
print(combined_df.head())